# Install extra packages

In [1]:
# install dependencies, if needed
# !pip install opendatasets
# !pip install pandas
# !pip install scprep
# !pip install binarytree

# Load training samples

In [2]:
# load libraries
from collections import Counter
import numpy as np
from scipy.stats import entropy
# import scprep
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn import datasets
from binarytree import build, build2, tree
import statistics as st

# this function duplicates a dataset with minor and
# random differences, to simulate data augmentation
def add_noise(df, noise_level=0.05):
  augmented_data = df.copy()
  for col in df.columns[:-1]:  # Excluindo a coluna de 'species'
    noise = np.random.normal(0, noise_level, df[col].shape)  # Ruído gaussiano
    augmented_data[col] = augmented_data[col] + noise
  return augmented_data

# load dataset
# wine = datasets.load_wine()
# features = wine['data']
# classes = wine['target']
# feature_names = wine['feature_names']
iris = datasets.load_iris()
features = iris.data
feature_names = iris.feature_names
classes_number = iris.target
class_names = iris.target_names
classes_with_names = [class_names[class_number] for class_number in classes_number]
class_colors = ['tab:green', 'tab:blue', 'tab:orange', 'tab:purple', 'tab:red']
header = feature_names + ['class']
body = np.column_stack((features, classes_number))
training_set = pd.DataFrame(body, columns=header)

# creates a new pandas Data Frame with original + 2x augmented data
augmented_training_set = pd.concat([training_set, 
                                    add_noise(training_set, noise_level=0.1), 
                                    add_noise(training_set, noise_level=0.1)], axis=0).reset_index(drop=True)

In [3]:
training_set

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),class
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0


In [4]:
# check that this augmented has 3x more elements
# than original dataset
augmented_training_set

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),class
0,5.100000,3.500000,1.400000,0.200000,0.0
1,4.900000,3.000000,1.400000,0.200000,0.0
2,4.700000,3.200000,1.300000,0.200000,0.0
3,4.600000,3.100000,1.500000,0.200000,0.0
4,5.000000,3.600000,1.400000,0.200000,0.0
...,...,...,...,...,...
445,6.666825,3.198263,5.283806,2.360055,2.0
446,6.343527,2.524797,5.099848,1.843829,2.0
447,6.664771,3.074464,5.221310,1.957131,2.0
448,6.243322,3.519539,5.435271,2.476572,2.0


# Decision Tree main functions

In [11]:
# this function computes the P 
def compute_p(counter):
  N = sum(counter.values())
  p = []
  for value in counter.values():
    p.append(value / N)
  return np.array(p)

# this function computes the gain, considering the application of a threshold
# using a certain feature and threshold
def compute_gain(all_features_counter, true_side_counter, false_side_counter):
  base = 2
  N = sum(all_features_counter.values())

  p_all = compute_p(all_features_counter)
  p_true = compute_p(true_side_counter)
  p_false = compute_p(false_side_counter)

  dict_true = dict(true_side_counter)
  true_proportion = sum(dict_true.values()) / N
  dict_false = dict(false_side_counter)
  false_proportion = sum(dict_false.values()) / N

  return entropy(p_all, base=base) - true_proportion * entropy(p_true, base=base) \
                                   - false_proportion * entropy(p_false, base=base)

# this function computes all possibilities of threshold values for
# a certain feature, from the minimum to the maximum valid values
def find_threshold(training_data, feature, round_number, show_figures = False, number_of_steps = 0):
  feature_vector = training_data[feature].values
  classes_vector = training_data[training_data.columns[-1]].values

  if number_of_steps == 0:
    # the thresholds vector is usually created based on real values
    thresholds_vector = np.unique(feature_vector.copy())
    thresholds_vector.sort()
  else:
    # we set a linspace only for the video recording purposes
    thresholds_vector = np.linspace(feature_vector.min(), feature_vector.max(), number_of_steps)
  gains_vector = np.zeros_like(thresholds_vector)

  best_threshold = thresholds_vector[0]
  best_gain = -np.inf
  i = 0
  for threshold in thresholds_vector:
    true_side = np.where(feature_vector > threshold, classes_vector, '')
    false_side = np.where(feature_vector <= threshold, classes_vector, '')

    true_side = np.delete(true_side, np.where(true_side == ''))
    false_side = np.delete(false_side, np.where(false_side == ''))

    gain = compute_gain(Counter(classes_vector),
                        Counter(true_side),
                        Counter(false_side))
    gains_vector[i] = gain

    if show_figures:
      matplotlib.rcParams.update({'font.size': 5})
      output_fig, ax_left = plt.subplots(figsize=(4, 2.5))
      for each_class in set(classes_vector):
        by_class_data = training_data[training_data['class'] == each_class]
        ax_left.hist(by_class_data[feature], 
                     label=class_names[int(each_class)], 
                     color=class_colors[int(each_class)], 
                     alpha=0.5)
      ax_left.set_ylabel('number of occurrences')
      ax_left.set_xlabel(f'feature {feature[:-5]}')   
      
      ax_right = ax_left.twinx()
      ax_right.plot(thresholds_vector[:i], gains_vector[:i], linewidth=line_width, color='red')
      ax_right.axvline(threshold, color='red', linewidth=line_width, label='current threshold')
      ax_right.set_ylim([0, 0.92])
      ax_right.set_ylabel('actual gain')

      h_right, l_right = ax_right.get_legend_handles_labels()
      h_left, l_left = ax_left.get_legend_handles_labels()
      ax_left.legend(handles=h_right + h_left, labels=l_right + l_left)

      plt.setp(ax_left.spines.values(), linewidth=line_width)
      plt.setp(ax_right.spines.values(), linewidth=line_width)
      ax_left.tick_params(width=line_width)
      ax_right.tick_params(width=line_width)
      ax_right.set_xlim([thresholds_vector[0], thresholds_vector[-1]])
      output_fig.tight_layout()
      output_fig.savefig(f'animation/find_threshold_round_{str(round_number).zfill(3)}_step_{str(i).zfill(3)}.png', format='png', dpi=200)
      plt.close()
    i = i + 1
    
    if gain != 0 and best_gain < gain:
      best_threshold = threshold
      best_gain = gain
  # print(gains_vector.max())
  return [best_threshold, best_gain]

# this function divides the data considering a certain
# feature and a certain threshold
def split_data(training_data, feature, threshold):
  true_side = training_data[training_data[feature] > threshold]
  false_side = training_data[training_data[feature] <= threshold]

  return [true_side, false_side]

# Call the algorithm and save steps for animation

In [12]:
# constants to control visualization and decision tree construction
line_width = 0.5
number_of_steps = 300
show_figures = True
min_cases = 50

# define input data
input_training_set = augmented_training_set # training_set
list_of_data = [input_training_set]
list_of_nodes = []
feature_names = input_training_set.columns[:-1]
round_number = 0
while len(list_of_data) > 0:
  actual_data = list_of_data.pop(0)
  if actual_data is None:
    list_of_nodes.append(None)
    continue

  if len(actual_data) > min_cases and len(set(actual_data['class'])) > 1:
    # find best feature/threshold for actual data
    best_gain = -np.inf
    best_feature = feature_names[0]
    best_threshold = -np.inf
    
    for feature in feature_names:
      [threshold, gain] = find_threshold(actual_data, feature, round_number, show_figures, number_of_steps)
      if show_figures:
        print(f'calling ffmpeg -y -f image2 -i "animation/find_threshold_round_{str(round_number).zfill(3)}_step_%3d.png" -r 25 -b 50000000 -filter:v "setpts=0.75*PTS" -an "decision_tree_round_{str(round_number).zfill(3)}.flv"')
        !ffmpeg -y -f image2 -i "animation/find_threshold_round_{str(round_number).zfill(3)}_step_%3d.png" -r 25 -b 50000000 -filter:v "setpts=0.75*PTS" -an "decision_tree_round_{str(round_number).zfill(3)}.flv"
      print(f'round {round_number} finished')
        
      round_number = round_number + 1
      if best_gain < gain:
        best_gain = gain
        best_feature = feature
        best_threshold = threshold
          
    list_of_nodes.append(f'{best_feature[:-5]} &gt; {best_threshold:2.3f}')
    [true_side, false_side] = split_data(actual_data, best_feature, best_threshold)

    if show_figures and False:
      plt.figure()
      for each_class in set(actual_data['class']):
        by_class_data = actual_data[actual_data['class'] == each_class]
        plt.hist(by_class_data[best_feature], label=each_class, color=class_colors[int(each_class)], alpha=0.5)
      plt.axvline(best_threshold, color='red')
      plt.title(best_feature[:-5])
      plt.legend(labels=class_names)
      plt.show()
      print(f'best feature is {best_feature}, threshold is {best_threshold}, with gain {best_gain}')

    list_of_data.append(true_side)
    list_of_data.append(false_side)
  else:
    majority_class = st.mode(actual_data['class'])
    list_of_nodes.append(class_names[int(majority_class)])
    list_of_data.append(None)
    list_of_data.append(None)

# display decision tree in simple text mode
decision_tree = build2(list_of_nodes)
print(decision_tree)

# export decision tree in PDF format
graph = decision_tree.graphviz()
graph.body
graph.render()

calling ffmpeg -y -f image2 -i "animation/find_threshold_round_000_step_%3d.png" -r 25 -b 50000000 -filter:v "setpts=0.75*PTS" -an "decision_tree_round_000.flv"


ffmpeg version 7.1-full_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 14.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libopenjpeg --enable-libquirc --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-libqrencode --enable-librav1e --enable-libsvtav1 --enable-libvvenc --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libvpx --enable-mediafoundation --enable-libass --enable-frei0r --enable-libfreety

round 0 finished
calling ffmpeg -y -f image2 -i "animation/find_threshold_round_001_step_%3d.png" -r 25 -b 50000000 -filter:v "setpts=0.75*PTS" -an "decision_tree_round_001.flv"


ffmpeg version 7.1-full_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 14.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libopenjpeg --enable-libquirc --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-libqrencode --enable-librav1e --enable-libsvtav1 --enable-libvvenc --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libvpx --enable-mediafoundation --enable-libass --enable-frei0r --enable-libfreety

round 1 finished
calling ffmpeg -y -f image2 -i "animation/find_threshold_round_002_step_%3d.png" -r 25 -b 50000000 -filter:v "setpts=0.75*PTS" -an "decision_tree_round_002.flv"


...

ffmpeg version 7.1-full_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 14.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libopenjpeg --enable-libquirc --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-libqrencode --enable-librav1e --enable-libsvtav1 --enable-libvvenc --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libvpx --enable-mediafoundation --enable-libass --enable-frei0r --enable-libfreety

round 18 finished
calling ffmpeg -y -f image2 -i "animation/find_threshold_round_019_step_%3d.png" -r 25 -b 50000000 -filter:v "setpts=0.75*PTS" -an "decision_tree_round_019.flv"


ffmpeg version 7.1-full_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 14.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libopenjpeg --enable-libquirc --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-libqrencode --enable-librav1e --enable-libsvtav1 --enable-libvvenc --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libvpx --enable-mediafoundation --enable-libass --enable-frei0r --enable-libfreety

round 19 finished

                                                        __________________________________________________________________________________________petal length &gt; 1.946__
                                                       /                                                                                                                   \
                       _____________________petal width &gt; 1.737_____________________                                                                                   setosa
                      /                                                                \
      ____petal length &gt; 4.800____                                    ____petal length &gt; 4.957_____________________
     /                               \                                  /                                                \
virginica                         virginica                        virginica                               _____petal width &

'Digraph.gv.pdf'